In [ ]:

import requests
import pandas as pd

#API Call
url = 'https://data.cityofchicago.org/resource/pk66-w54g.json?$limit=125000'
r = requests.get(url)
response_dict = r.json()

if r.status_code != 200:
    df = pd.read_csv("https://raw.githubusercontent.com/LakshmiPriyaDiwakar2706/IS445-Final-Proj-Part-1/main/Chicago_Park_District_-_Event_Permits_20240411.csv")
else:
    df = pd.DataFrame(response_dict)

# DATA PREPROCESSING

data_copy = df.copy()
data_copy.head()
data_copy['reservation_start_date'] = pd.to_datetime(data_copy['reservation_start_date'])
data_copy['reservation_end_date'] = pd.to_datetime(data_copy['reservation_end_date'])
data_copy['reservation_start_date']
data_copy['reservation_end_date']
## Handling missing values
data_copy['park_number'] = data_copy['park_number'].fillna(-1)


### Events per year
data_copy['Year'] = data_copy['reservation_start_date'].dt.year
events_per_year = data_copy['Year'].value_counts().sort_index()
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Event 6/10,000+':'Event 6'})
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Festival/Performance/12,001+ Event':'Festival/Performance Event'})
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Parking Lot Rental':'Parking Lot Rental Permit'})
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Promotions':'Promotional Event'})
mappings = {
'Administrative Reservation':'Administrative Reservation',
'Commemorative':'Commemorative',
'Athletic':'Athletic',
'Corporate Event':'Corporate Event',
'Media':'Media',
'Picnic':'Picnic',
'Venue Rental':'Venue Rental',
'Event 1 Cluster':'Event 1',
'Event 2 Cluster':'Event 2',
'Event 3 Cluster':'Event 3',
'Event 4 Cluster':'Event 4',
'Event 5 Cluster':'Event 5'
}
def categorize_event(event_description):
    for key,values in mappings.items():
        if key in event_description:
            return values
    return event_description
data_copy['Event Type Category'] = data_copy['event_type'].apply(categorize_event)



#Renaming the columns to avoid duplications
mappings = {
'Approved':'Approved',
'Completed':'Approved',
'Issued':'Approved'
}
def status_change(permit_status):
    for key,values in mappings.items():
        if key in permit_status:
            return values
    return permit_status
data_copy['permit_status'] = data_copy['permit_status'].apply(status_change)


#DASHBOARD

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors
from bqplot import pyplot as bq_plt, LinearScale, Axis, Lines, Figure, OrdinalScale, Scatter, Tooltip
from ipywidgets import Dropdown, Button, HBox, VBox, Output, Layout, Image, Label, GridBox, HTML
from IPython.display import display


# Define CSS styles for bqplot
bq_plt.style = {'title-font-size': '20px', 'axis-label-font-size': '15px', 'title-align': 'center'}

# Setup data
approved_events = data_copy[data_copy['permit_status'] == 'Approved'].copy()
approved_events['Year'] = pd.to_datetime(approved_events['reservation_start_date']).dt.year
approved_events['Month'] = pd.to_datetime(approved_events['reservation_start_date']).dt.month
approved_events['Month_Name'] = pd.to_datetime(approved_events['reservation_start_date']).dt.strftime('%b')

# Function to categorize months into seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

approved_events['Season'] = approved_events['Month'].apply(get_season)
heatmap_data = approved_events.pivot_table(index='park_number', columns='Season', values='reservation_start_date', aggfunc='count', fill_value=0)

# Create scales for the bqplot
x_sc = OrdinalScale()
y_sc = LinearScale()

# Create the line and scatter plot for year data
line = Lines(scales={'x': x_sc, 'y': y_sc}, colors=['#f71414'])
scatter = Scatter(scales={'x': x_sc, 'y': y_sc}, colors=['#f71414'],
                  tooltip=Tooltip(fields=['x', 'y'], labels=['Year', 'Number of Events']), unhovered_style={'opacity': 0.5})

# Create the axes
x_ax = Axis(label='Year', scale=x_sc)
y_ax = Axis(label='Number of Events', scale=y_sc, orientation='vertical')

# Create the figure
fig = Figure(marks=[line, scatter], axes=[x_ax, y_ax], title='Number of Events per Year (Approved Permits)')
fig.layout.width = '600px'
fig.layout.height = '400px'


# Additional output widget for the donut chart
donut_chart_output = Output()

def update_donut_chart(year):
    if year != 'All Years':
        filtered_data = approved_events[approved_events['Year'] == int(year)]
    else:
        filtered_data = approved_events

    season_data = filtered_data.groupby('Season')['reservation_start_date'].count()
    colors = ["lightgrey", "darkgrey", "#f71414", "grey"]
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_red_grey", colors, N=256)

    with donut_output:
        donut_output.clear_output(wait=True)
        fig, ax = plt.subplots()
        outer_circle = ax.pie(season_data, labels=season_data.index, autopct='%1.1f%%', startangle=90, colors=colors, wedgeprops=dict(width=0.3, edgecolor='w'))
        inner_circle = ax.pie([1], radius=0.5, colors=['white'])  # Draw a white circle at the center
        ax.set_aspect('equal')
        ax.set_title('Total Reservations by Season')
        plt.show()

bar_chart_output = Output()

def update_bar_chart(year):
    if year != 'All Years':
        filtered_data = approved_events[approved_events['Year'] == int(year)]
    else:
        filtered_data = approved_events
    top_parks = filtered_data['park_number'].value_counts().nlargest(5).index
    top_parks_data = filtered_data[filtered_data['park_number'].isin(top_parks)]
    top_requestors = top_parks_data.groupby(['park_number', 'requestor_']).size().reset_index(name='counts')
    top_requestors = top_requestors.sort_values(by=['park_number', 'counts'],ascending=[True, False])    
    top_requestors_by_park = top_parks_data.groupby(['park_number', 'requestor_']).size().reset_index(name='counts')
    total_reservations_by_requestor = top_requestors_by_park.groupby('requestor_').sum('counts').nlargest(10, 'counts')
    top_requestors_filtered = top_requestors_by_park[top_requestors_by_park['requestor_'].isin(total_reservations_by_requestor.index)]
    top_requestors_filtered = top_requestors_filtered.sort_values(by=['counts'],ascending=False)


    with bar_output:
        bar_output.clear_output(wait=True)
        colors = ["#f71414", "gray", "lightgray"]
        cmap = mcolors.LinearSegmentedColormap.from_list("custom_red_grey", colors, N=100)
        plt.figure(figsize=(12, 5))
        sns.barplot(x='counts', y='requestor_', hue='park_number', data=top_requestors_filtered, dodge=False,
                    palette=[cmap(i / float(len(top_parks))) for i in range(len(top_parks))],
                    linewidth=1, saturation=1, alpha=1, ci=None, orient='h', capsize=0, errwidth=0)
        plt.legend(title='Park Number', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title('Top 10 Requestors with Park Number (Approved Events Only)')
        plt.xlabel('Number of Reservations')
        plt.ylabel('Requestor Name')
        plt.tight_layout()
        plt.show()

        
line_chart_output = Output()
        
def update_line_chart(year):
    # Reset axes scales for consistent updates
    global x_sc, x_ax
    if year != 'All Years':
        monthly_data = approved_events[approved_events['Year'] == int(year)].groupby('Month_Name')['reservation_start_date'].count().reindex(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fill_value=0)
    else:
        yearly_data = approved_events.groupby('Year')['reservation_start_date'].count()

    with line_chart_output:
        line_chart_output.clear_output(wait=True)
        if year != 'All Years':
            line.x = monthly_data.index
            line.y = monthly_data.values
            scatter.x = monthly_data.index
            scatter.y = monthly_data.values
            x_ax.label = 'Month'
        else:
            line.x = yearly_data.index
            line.y = yearly_data.values
            scatter.x = yearly_data.index
            scatter.y = yearly_data.values
            x_ax.label = 'Year'
        
        
        x_sc = OrdinalScale()
        fig.axes = [x_ax, y_ax]
        display(fig)
    
# Output widget for the heatmap
heatmap_output = Output()

def update_heatmap(year):
    # Filtering the data for the selected year or use all data if 'All Years' is selected
    filtered_data = approved_events[approved_events['Year'] == int(year)] if year != 'All Years' else approved_events
    
    # Creating a pivot table for the heatmap data
    heatmap_data = filtered_data.pivot_table(index='park_number', columns='Season', values='reservation_start_date', aggfunc='count', fill_value=0)

    # Calculating the total reservations for each park and get the top 10 parks
    total_reservations = heatmap_data.sum(axis=1).nlargest(10)
    top_parks = total_reservations.index

    # Filtering the heatmap data to include only the top 10 parks
    heatmap_data = heatmap_data.loc[top_parks]

    # If 'park_facility_name' is a column in 'filtered_data', create a mapping dictionary
    park_names = pd.Series(filtered_data['park_facility_name'].values, index=filtered_data['park_number']).to_dict()

    # Replacing park numbers with names in the index of heatmap data
    heatmap_data.index = heatmap_data.index.map(lambda x: park_names.get(x, 'Unknown'))

    # Creating a custom colormap from light grey to red
    colors = ["white", "red"]  # lightgrey to red
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_grey_red", colors)

    with heatmap_output:
        heatmap_output.clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        ax = sns.heatmap(heatmap_data, annot=True, fmt="d", cmap=cmap)
        plt.title('Top 10 Parks: Reservations by Season')
        plt.xlabel('Season')
        plt.ylabel('Park Name')
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)  # Make sure park names are readable
        plt.show()

    

def update_plots(year):
    update_heatmap(year)
    update_line_chart(year)
    update_donut_chart(year)  
    update_bar_chart(year)  

# Widgets
year_dropdown = Dropdown(options=['All Years'] + sorted(approved_events['Year'].unique().astype(str).tolist()), description='Select Year')
year_dropdown.observe(lambda change: update_plots(change.new), names='value')

def refresh_dashboard(_):
    year_dropdown.value = 'All Years'
    update_plots('All Years')
refresh_button = Button(description="Refresh", button_style='danger', icon='refresh')
refresh_button.on_click(refresh_dashboard)
refresh_button.style.button_color = 'Red'



# Control Box
control_box = HBox([year_dropdown, refresh_button], layout=Layout(justify_content='space-between', width='100%'))

# Outputs
heatmap_output = Output()
donut_output = Output()
bar_output = Output()

# Initial update
update_plots('All Years')

# Grid layout for dashboard
dashboard = GridBox(children=[control_box, fig, heatmap_output, donut_output, bar_output],
                     layout=Layout(
                         width='100%',
                         grid_template_rows='auto auto auto',
                         grid_template_columns='50% 50%',
                         grid_template_areas='''
                         "header header"
                         "left right"
                         "lower lower"
                         '''
                     ))
# Assign grid areas
control_box.layout.grid_area = 'header'
fig.layout.grid_area = 'left'
line_chart_output.layout.grid_area = 'left'
heatmap_output.layout.grid_area = 'right'
donut_chart_output.layout.grid_area = 'lower right'
bar_chart_output.layout.grid_area = 'lower left'

# Display the dashboard
dashboard


In [22]:

import requests
import pandas as pd

#API Call
url = 'https://data.cityofchicago.org/resource/pk66-w54g.json?$limit=125000'
r = requests.get(url)
response_dict = r.json()

if r.status_code != 200:
    df = pd.read_csv("https://raw.githubusercontent.com/LakshmiPriyaDiwakar2706/IS445-Final-Proj-Part-1/main/Chicago_Park_District_-_Event_Permits_20240411.csv")
else:
    df = pd.DataFrame(response_dict)

# DATA PREPROCESSING

data_copy = df.copy()
data_copy.head()
data_copy['reservation_start_date'] = pd.to_datetime(data_copy['reservation_start_date'])
data_copy['reservation_end_date'] = pd.to_datetime(data_copy['reservation_end_date'])
data_copy['reservation_start_date']
data_copy['reservation_end_date']
## Handling missing values
data_copy['park_number'] = data_copy['park_number'].fillna(-1)


### Events per year
data_copy['Year'] = data_copy['reservation_start_date'].dt.year
events_per_year = data_copy['Year'].value_counts().sort_index()
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Event 6/10,000+':'Event 6'})
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Festival/Performance/12,001+ Event':'Festival/Performance Event'})
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Parking Lot Rental':'Parking Lot Rental Permit'})
data_copy['event_type'] = data_copy['event_type'].replace({'Permit - Promotions':'Promotional Event'})
mappings = {
'Administrative Reservation':'Administrative Reservation',
'Commemorative':'Commemorative',
'Athletic':'Athletic',
'Corporate Event':'Corporate Event',
'Media':'Media',
'Picnic':'Picnic',
'Venue Rental':'Venue Rental',
'Event 1 Cluster':'Event 1',
'Event 2 Cluster':'Event 2',
'Event 3 Cluster':'Event 3',
'Event 4 Cluster':'Event 4',
'Event 5 Cluster':'Event 5'
}
def categorize_event(event_description):
    for key,values in mappings.items():
        if key in event_description:
            return values
    return event_description
data_copy['Event Type Category'] = data_copy['event_type'].apply(categorize_event)



#Renaming the columns to avoid duplications
mappings = {
'Approved':'Approved',
'Completed':'Approved',
'Issued':'Approved'
}
def status_change(permit_status):
    for key,values in mappings.items():
        if key in permit_status:
            return values
    return permit_status
data_copy['permit_status'] = data_copy['permit_status'].apply(status_change)


#DASHBOARD

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors
from bqplot import pyplot as bq_plt, LinearScale, Axis, Lines, Figure, OrdinalScale, Scatter, Tooltip
from ipywidgets import Dropdown, Button, HBox, VBox, Output, Layout, Image, Label, GridBox, HTML
from IPython.display import display


# Define CSS styles for bqplot
bq_plt.style = {'title-font-size': '20px', 'axis-label-font-size': '15px', 'title-align': 'center'}

# Setup data
approved_events = data_copy[data_copy['permit_status'] == 'Approved'].copy()
approved_events['Year'] = pd.to_datetime(approved_events['reservation_start_date']).dt.year
approved_events['Month'] = pd.to_datetime(approved_events['reservation_start_date']).dt.month
approved_events['Month_Name'] = pd.to_datetime(approved_events['reservation_start_date']).dt.strftime('%b')

# Function to categorize months into seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

approved_events['Season'] = approved_events['Month'].apply(get_season)
heatmap_data = approved_events.pivot_table(index='park_number', columns='Season', values='reservation_start_date', aggfunc='count', fill_value=0)

# Create scales for the bqplot
x_sc = OrdinalScale()
y_sc = LinearScale()

# Create the line and scatter plot for year data
line = Lines(scales={'x': x_sc, 'y': y_sc}, colors=['#f71414'])
scatter = Scatter(scales={'x': x_sc, 'y': y_sc}, colors=['#f71414'],
                  tooltip=Tooltip(fields=['x', 'y'], labels=['Year', 'Number of Events']), unhovered_style={'opacity': 0.5})

# Create the axes
x_ax = Axis(label='Year', scale=x_sc)
y_ax = Axis(label='Number of Events', scale=y_sc, orientation='vertical',label_offset='50px')

# Create the figure
fig = Figure(marks=[line, scatter], axes=[x_ax, y_ax], title='Number of Events per Year (Approved Permits)')
fig.layout.width = '540px'
fig.layout.height = '400px'


# Additional output widget for the donut chart
donut_chart_output = Output()

def update_donut_chart(year):
    if year != 'All Years':
        filtered_data = approved_events[approved_events['Year'] == int(year)]
    else:
        filtered_data = approved_events

    season_data = filtered_data.groupby('Season')['reservation_start_date'].count()
    colors = ["lightgrey", "darkgrey", "#f71414", "grey"]
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_red_grey", colors, N=256)

    with donut_output:
        donut_output.clear_output(wait=True)
        fig, ax = plt.subplots()
        outer_circle = ax.pie(season_data, labels=season_data.index, autopct='%1.1f%%', startangle=90, colors=colors, wedgeprops=dict(width=0.3, edgecolor='w'))
        inner_circle = ax.pie([1], radius=0.5, colors=['white'])  # Draw a white circle at the center
        ax.set_aspect('equal')
        ax.set_title('Total Reservations by Season')
        plt.show()

bar_chart_output = Output()

def update_bar_chart(year):
    if year != 'All Years':
        filtered_data = approved_events[approved_events['Year'] == int(year)]
    else:
        filtered_data = approved_events
    top_parks = filtered_data['park_number'].value_counts().nlargest(5).index
    top_parks_data = filtered_data[filtered_data['park_number'].isin(top_parks)]
    top_requestors = top_parks_data.groupby(['park_number', 'requestor_']).size().reset_index(name='counts')
    top_requestors = top_requestors.sort_values(by=['park_number', 'counts'],ascending=[True, False])    
    top_requestors_by_park = top_parks_data.groupby(['park_number', 'requestor_']).size().reset_index(name='counts')
    total_reservations_by_requestor = top_requestors_by_park.groupby('requestor_').sum('counts').nlargest(10, 'counts')
    top_requestors_filtered = top_requestors_by_park[top_requestors_by_park['requestor_'].isin(total_reservations_by_requestor.index)]
    top_requestors_filtered = top_requestors_filtered.sort_values(by=['counts'],ascending=False)


    with bar_output:
        bar_output.clear_output(wait=True)
        colors = ["#f71414", "gray", "lightgray"]
        cmap = mcolors.LinearSegmentedColormap.from_list("custom_red_grey", colors, N=100)
        plt.figure(figsize=(12, 5))
        sns.barplot(x='counts', y='requestor_', hue='park_number', data=top_requestors_filtered, dodge=False,
                    palette=[cmap(i / float(len(top_parks))) for i in range(len(top_parks))],
                    linewidth=1, saturation=1, alpha=1, ci=None, orient='h', capsize=0, errwidth=0)
        plt.legend(title='Park Number', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title('Top 10 Requestors with Park Number (Approved Events Only)')
        plt.xlabel('Number of Reservations')
        plt.ylabel('Requestor Name')
        plt.tight_layout()
        plt.show()

        
line_chart_output = Output()
        
def update_line_chart(year):
    # Reset axes scales for consistent updates
    global x_sc, x_ax
    if year != 'All Years':
        monthly_data = approved_events[approved_events['Year'] == int(year)].groupby('Month_Name')['reservation_start_date'].count().reindex(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], fill_value=0)
    else:
        yearly_data = approved_events.groupby('Year')['reservation_start_date'].count()

    with line_chart_output:
        line_chart_output.clear_output(wait=True)
        if year != 'All Years':
            line.x = monthly_data.index
            line.y = monthly_data.values
            scatter.x = monthly_data.index
            scatter.y = monthly_data.values
            x_ax.label = 'Month'
        else:
            line.x = yearly_data.index
            line.y = yearly_data.values
            scatter.x = yearly_data.index
            scatter.y = yearly_data.values
            x_ax.label = 'Year'
        
        
        x_sc = OrdinalScale()
        fig.axes = [x_ax, y_ax]
        display(fig)
    
# Output widget for the heatmap
heatmap_output = Output()

def update_heatmap(year):
    # Filtering the data for the selected year or use all data if 'All Years' is selected
    filtered_data = approved_events[approved_events['Year'] == int(year)] if year != 'All Years' else approved_events
    
    # Creating a pivot table for the heatmap data
    heatmap_data = filtered_data.pivot_table(index='park_number', columns='Season', values='reservation_start_date', aggfunc='count', fill_value=0)

    # Calculating the total reservations for each park and get the top 10 parks
    total_reservations = heatmap_data.sum(axis=1).nlargest(10)
    top_parks = total_reservations.index

    # Filtering the heatmap data to include only the top 10 parks
    heatmap_data = heatmap_data.loc[top_parks]

    # If 'park_facility_name' is a column in 'filtered_data', create a mapping dictionary
    park_names = pd.Series(filtered_data['park_facility_name'].values, index=filtered_data['park_number']).to_dict()

    # Replacing park numbers with names in the index of heatmap data
    heatmap_data.index = heatmap_data.index.map(lambda x: park_names.get(x, 'Unknown'))

    # Creating a custom colormap from light grey to red
    colors = ["white", "red"]  # lightgrey to red
    cmap = mcolors.LinearSegmentedColormap.from_list("custom_grey_red", colors)

    with heatmap_output:
        heatmap_output.clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        ax = sns.heatmap(heatmap_data, annot=True, fmt="d", cmap=cmap)
        plt.title('Top 10 Parks: Reservations by Season')
        plt.xlabel('Season')
        plt.ylabel('Park Name')
        ax.set_yticklabels(ax.get_yticklabels(), rotation=0)  # Make sure park names are readable
        plt.show()

    

def update_plots(year):
    update_heatmap(year)
    update_line_chart(year)
    update_donut_chart(year)  
    update_bar_chart(year)  

# Widgets
year_dropdown = Dropdown(options=['All Years'] + sorted(approved_events['Year'].unique().astype(str).tolist()), description='Select Year')
year_dropdown.observe(lambda change: update_plots(change.new), names='value')

def refresh_dashboard(_):
    year_dropdown.value = 'All Years'
    update_plots('All Years')
refresh_button = Button(description="Refresh", button_style='danger', icon='refresh')
refresh_button.on_click(refresh_dashboard)
refresh_button.style.button_color = 'Red'





# Outputs
heatmap_output = Output()
donut_output = Output()
bar_output = Output()

# Initial update
update_plots('All Years')



title_html = HTML(
    "<h2 style='text-align: center; width: 100%;'>Chicago Parks Reservations Dashboard</h2>",
    layout=Layout(height='auto', width='100%')
)


#  control_box to include the title and align items
control_box = HBox(
    [title_html, year_dropdown, refresh_button],
    layout=Layout(
        width='100%',
        display='flex',
        justify_content='space-between',  
        align_items='center',  
        margin='0px 0px 15px 0px'  
    )
)




# control_box = HBox([year_dropdown, refresh_button], layout=Layout(width='100%', display='flex', justify_content='flex-end', margin='0px 0px 15px 0px'))

donut_chart_output.layout = Layout(margin='0px 65px 0px 0px')
heatmap_output.layout = Layout(margin='40px 0px 0px 0px')

lower_left = HBox(
    [donut_chart_output],
    layout=Layout(
        width='100%',
        display='flex',
        flex_flow='row nowrap',
        justify_content='space-between', 
        align_items='center'
    )
)

# Grid layout for dashboard
dashboard = GridBox(children=[control_box, line_chart_output, heatmap_output, donut_output, bar_output],
                     layout=Layout(
                         width='100%',
                         grid_template_rows='auto auto auto',
                         grid_template_columns='50% 50%',
                         grid_template_areas='''
                         "header header"
                         "left right"
                         "lower lower"
                         '''
                     ))
# Assign grid areas
control_box.layout.grid_area = 'header'
line_chart_output.layout.grid_area = 'left'
heatmap_output.layout.grid_area = 'right'
donut_chart_output.layout.grid_area = 'lower right'
bar_chart_output.layout.grid_area = 'lower left'

# Display the dashboard
dashboard


GridBox(children=(HBox(children=(HTML(value="<h2 style='text-align: center; width: 100%;'>Chicago Parks Reserv…